In [ ]:
from importlib import reload
reload(ex)
reload(an)
reload(ocr)
reload(plt)

### import library

In [ ]:
import os
import extractor as ex
import annotator as an
import ocr
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
### class dependent
email_path = 'emails/2023FMath104A-email.csv'

### quiz dependent
quiz_name = '1112_1' ### 登記成績用
key_name = "1112_1_key.csv" ### 使用的答案

### file dependent
proj_name = "1112_1" ### 檔名去掉 .pdf
# full_path = '/home/jephian/Downloads/1102_2m_4_full.csv'
score_list = [email_path, quiz_name] #+ [quiz_name + 'm_%s'%i for i in range(1,2)]

#### 1. Extract pdf file to get checkcode, qrcode and whole paper for grading by extractor

In [ ]:
ex.extract(path=os.path.join('pdfs','%s.pdf'%proj_name), 
           mode='grade', 
           key_path=os.path.join('keys', key_name), 
           box='auto', ### 'auto' or (left, top, right, bottom)
          )
raw = ex.raw_data(proj_name)
fig = raw.examine(0,min(25,raw.num))
print(raw.num)

In [ ]:
raw.examine()

#### 2. Use OCR system to classify the digits in checkcode and check the results

In [ ]:
ocr.predict(raw, 
            mdl_path='svc-1639.joblib')

In [ ]:
raw.labeler(patch=os.path.join(proj_name, '%s_pred.csv'%proj_name), remove=False)

Manually mark invalid answers with `d` .  
Record the accuracy if you wish.

In [ ]:
### move the patch to patches folder
os.rename(os.path.join(proj_name, '%s_patch.csv'%proj_name), 
          os.path.join('patches', '%s_patch.csv'%proj_name))

#### 3. Import edit full.csv

In [ ]:
raw = ex.raw_data(proj_name)
print(raw.num)
paper = an.raw_paper(proj_name)
print(paper.num)

In [ ]:
paper.full.head()

Edit the id column manually.  
Check qr code if necessary.

In [ ]:
### reload
raw = ex.raw_data(proj_name)
print(raw.num)
paper = an.raw_paper(proj_name)
print(paper.num)

### get cor_ans
paper.get_cor_ans(key_path=os.path.join('keys', key_name))

In [ ]:
paper.full.head(None)

In [ ]:
### get label from raw
paper.get_label()

### get points
paper.grade()

### get email addresses
paper.get_email(email_path)

#### 4. Auto-grading on papers by annotator

In [ ]:
raw_paper = an.raw_paper(proj_name)
raw_paper.annotate()

#### 5. Send emails and record

In [ ]:
paper = an.raw_paper(proj_name)
print(paper.num)

In [ ]:
### do dry, test, and then send
paper.group_email(mode='send') 

Stage keys, patches on GitHub.  
Put emails, pdfs, full.csv on Google Drive.  

Collect all records

In [ ]:
record = an.get_record(*score_list)

In [ ]:
print(record.shape)
record.head()

In [ ]:
### see who miss the mandatory quiz
record.loc[record.filename == 'miss']

Change `miss` to `ignore` for students with particular reason.

In [ ]:
### add ignore
ignore_list = [25]
for ind in ignore_list:
    record.loc[ind, 'filename'] = 'ignore'

In [ ]:
### save file
record.to_csv(score_list[1]+'_record.csv')

In [ ]:
scores = an.get_scores(record, quiz_name)

In [ ]:
print(scores.shape)
scores.head(None)

Update scores on Google Drive.  
Announce on Cyber University.  
Put `_record` and `_scores` in the course folder `quiz-records`.